# Introduction

`geneanot` provides eukaryotes gene annotation based on [Ensembl](https://www.ensembl.org/index.html).

## Species
`geneanot` supports all eukaryotes species. The default species is Homo sapiens.

## Requirements
`geneanot` requires an Ensembl annotation GFF3 file. `geneanot` provides functions to download the Ensembl annotation file to a local folder, which will be used by the package. If a local annotation file exists, `geneanot` can also check if there is a new version in Ensembl and download it.

The user must designate a folder that holds the Ensembl annotation file (or that will be used by `geneanot` to download the Ensembl annotation file to). This is defined below by the variable `Annotation_folder`.

## Chromosome Data
Some of the provided methods require access to the chromosome data. The two supported access modes are:
1. `local` - using a local chromosome Fasta file provided by the user. Or,
2. `remote` - using remote Ensembl chromosome files

The `local` mode requires the user to provide the corresponding chromosome Fasta file (this can be done at instantiation, and/or during the lifetime of the annotation object). The Fasta file **must** contain equal number of bps per row for all sequence rows (except possibly the last row), since this is assumed by `geneanot` to support fast random access query. The chromosome Fasta files can be downloaded from Ensembl. For example, for Homo spaiens, these are located in
`https://ftp.ensembl.org/pub/current_fasta/homo_sapiens/dna/` (the required files are `Homo_sapiens.GRCh38.dna_sm.chromosome.<chromosome number>.fa.gz`, e.g. `Homo_sapiens.GRCh38.dna_sm.chromosome.1.fa.gz` for chromosome 1). The Ensembl chromosome Fasta files contain equal number of bps per row for all sequence rows.
Since in `local` mode `geneanot` uses a local chromosome Fasta file, the sequence retrival is fast.

In the `remote` mode, `geneanot` uses the Ensembl REST API to extract sequences from the corresponding chromosome. This implies a slower implementation, and requires a working network connection.

The access mode is determined by the variable `chrm_fasta_file` below. Setting it to `None` implies `remote` mode, otherwise, holding a valid chromosome Fasta file name implies `local` mode. Note that this may be set upon instantiating the annotation class, and/or during the lifetime of the annotation object.


The following sections provide useful usage examples.



In [1]:
from pathlib import Path
import geneanot as u

# folder that holds (or will hold) the Ensembl annotation file.
Annotation_folder: Path = Path('./../AnnotationDB')

# Human Gene Annotation

We start with examples of annotating Homo sapiens genes and transcripts.

## Informative
This is not required for actual annotation. It just provides information about the local and remote (Ensembl) annotation files.

In [ ]:
# Check for Ensembl annotation files in the local folder
if (local_files := u.get_annotation_files_info_in_folder(Annotation_folder)):
    versions = list(local_files.keys())
    print(f"Found {len(local_files)} annotation files in {Annotation_folder}. {versions=}, maximal version = {max(versions)}.")
else:
    print(f"No annotation file in {Annotation_folder}.")

# check Ensembl latest version
ensembl_version = u.get_ensembl_release()
print(f"Ensembl latest version is {ensembl_version}.")

## Annotate

### Initialize the Annotation Folder
This is required if:
- the local annotation folder does not contain an annotation file, as it will be downloaded from the Ensembl site, or
- we want to update the local annotation file to the lastest annotation file

Otherwise, this can be ignored, and `annotation_full_file` must be set by the user, and must point to a valid annotation file name. 

In [ ]:
enable_download: bool = False  # enables/disables annotation file download from Ensembl only when local version < ensembl version.
                               # If Annotation_folder does not contain anotation file, the download is done regardless of this setting.
# ------------------------------------------------------------------------------------------------------------------------------------
download_done, ensembl_file, local_file = u.update_local_release_to_latest(Annotation_folder, enable_download=enable_download)
annotation_full_file = Annotation_folder / (ensembl_file if download_done else local_file)
print(f"Annotation file to use: {annotation_full_file}.\n")

### Instantiate the Annotation Class

The instantiation is slow, as `geneanot` starts by preparing several data structures from the annotation file. We will see later another (faster) way to instantiate the annotation class, which is more relevant in cases where we are interested in annotating many genes.

In [ ]:
gene: str = 'EGFR'  # gene name or gene ID
verbose: bool = True  # set to True to get informative prints

species: str = 'homo_sapiens'
# -----------------------------------------------------------
# instantiate in remote access mode
gA = u.Gene_cls(gene, annotation_full_file, species=species, verbose=verbose)

# Or - instantiate in local access mode
# chrm_fasta_file: str = './../Chromosome/Homo_sapiens.GRCh38.dna_sm.chromosome.7.fa'
# gA = u.Gene_cls(gene, annotation_full_file, species=species, chrm_fasta_file=chrm_fasta_file, verbose=verbose)

# regardless, later we can set gA.chrm_fasta_file to either None or a valid chromosome Fasta
# file name to set the mode to remote or local, respectively.

### General Gene Information

In [ ]:
# print general gene information
gA.info()

print(f"\n{gene} contains {len(gA)} transcripts.")
gA.show_all_transcript_IDs()

print(f"\nGene start = {gA.gene_start:,}, gene end = {gA.gene_end:,}.")

print(f"\n{gene=} encoded on the {'negative' if gA.rev else 'positive'} DNA strand.")

In [ ]:
# the various gene attributes printed above are also accessible by gA
print(gA.gene_name, gA.gene_ID, gA.gene_desc, gA.gene_type, gA.gene_ver, gA.rev, gA.chrm, gA.gene_start, gA.gene_end, len(gA.transcripts_info), sep='\n')


### Transcript Annotation

Here we annotate a single transcript of the gene.

In [ ]:
transcript_id: str = 'ENST00000275493'
# -----------------------------------
# print transcript information
gA.transcript_info(transcript_id, verbose=True)

# the transcript features are accessible via gA.transcripts_info[transcript_id], for example
t_info = gA.transcripts_info[transcript_id]
print('\n', t_info['transcript_name'], t_info['transcript_v_id'], t_info['transcript_ver'])

# all available features (i.e., t_info[feature])
available_transcript_features = ''.join(f"{i}. {feature}\n" for i, feature in zip(range(1, len(t_info)+1), t_info.keys()))
print(f"\nThe transcript accessible features are:\n{available_transcript_features}")


Regarding the exon phases: 
- exon end phase is the exon phase of the next exon.
- conversion from exon phase to CDS phase: 0-->0, 1-->2, 2-->1.

In [ ]:
# Transcript table - exon and intron table.
df = gA.exon_intron_map(transcript_id)
display(df)

# the transcript table can be written to excel, CSV or HTML file
gA.exon_intron_map_to_csv(transcript_id, './../Reports/transcript_table.csv')
gA.exon_intron_map_to_excel(transcript_id, './../Reports/transcript_table.xlsx', usr_desc={"Description": "Transcript table", "Transcript": transcript_id})
gA.exon_intron_map_to_html(transcript_id, './../Reports/transcript_table.html')

In [ ]:
# mRNA table - maps the ORF to the mRNA sequence.
df = gA.exon_map(transcript_id)
display(df)

# similarly to exon_intron_map() above, this table can also we written to excel, CSV or HTML file
gA.exon_map_to_csv(transcript_id, './../Reports/mRNA_table.csv')
gA.exon_map_to_excel(transcript_id, './../Reports/mRNA_table.xlsx', usr_desc={"Description": "mRNA table", "Transcript": transcript_id})
gA.exon_map_to_html(transcript_id, './../Reports/mRNA_table.html')

#### Sequences

Here we retrieve different sequences (e.g., mRNA sequence, AA sequence, etc.).

In [ ]:
# The access mode can be set/changed anytime:

# set this to use local mode (if instantiation was done with remote mode)
gA.chrm_fasta_file = f'./../Chromosome/Homo_sapiens.GRCh38.dna_sm.chromosome.{gA.chrm}.fa'

# Or, set this to use remote mode (if instantiation was with local mode)
# gA.chrm_fasta_file = None

print(f"Access mode is {gA.access_mode()}.")

In [ ]:
pre_mRNA_seq = gA.seq(transcript_id).upper()
print(f"The pre-mRNA sequence contains {len(pre_mRNA_seq):,} bps.")

rna_seq = gA.rna(transcript_id).upper()
print(f"\nrna:\n{rna_seq}\n{len(rna_seq):,} bps.")

orf_seq = gA.ORF(transcript_id).upper()
print(f"\norf:\n{orf_seq}\n{len(orf_seq):,} bps.")

aa_seq = gA.AA(transcript_id)
print(f"\nprotein:\n{aa_seq}\n{len(aa_seq):,} AAs.")

utr5_seq = gA.UTR5(transcript_id).upper()
print(f"\n5'UTR:\n{utr5_seq}\n{len(utr5_seq):,} bps.")

utr3_seq = gA.UTR3(transcript_id).upper()
print(f"\n3'UTR=\n{utr3_seq}\n{len(utr3_seq):,} bps.")

# An Exon sequence
exon_number: int = 5
exon_seq, seq_info = gA.exon_intron_seq('Exon', exon_number, transcript_id)
print(f"\nExon #{exon_number}:\n{exon_seq}\n{seq_info}")

# An Intron sequence
intron_number: int = 5
intron_seq, seq_info = gA.exon_intron_seq('Intron', intron_number, transcript_id)
print(f"\nIntron #{intron_number} contains {len(intron_seq):,} bps.\n{seq_info.upper()}")

# modified transcript - based on a list of exons and introns
use_exon_list: list[int] = [1, 3]  # exon numbers to use
use_intron_list: list[int] = [2]  # intron numbers to use
m_seq = gA.modified_transcript(use_exon_list, use_intron_list, transcript_id)
print(f"\nA modified transcript containing exons {', '.join(map(str,use_exon_list))} and introns {', '.join(map(str,use_intron_list))} contains {len(m_seq.upper()):,} bps.")

#### Queries
All positions below are 1-based positions (unless otherwise indicated).

In [ ]:
# query a start and a stop codon positions in the chromosome and the RNA
start_codon_info, stop_codon_info = gA.start_and_stop_codons_pos(transcript_id)
# each _info contains (chromosome_position_of_first_bp_of_codon, RNA_position_of_first_bp_of_codon)
print(f"{start_codon_info=}\n{stop_codon_info=}")

In [ ]:
# query a start and an end ORF positions in the exons
# (i.e., the exon index and the offset within the exon of the 
# first bp of the start codon and the last bp of the stop codon)
orf_start_end_info = gA.orf_start_and_end_exon_info(transcript_id)
# here, 'exon_number' value is 1-based, 'exon_offset' is 0-based
print(orf_start_end_info)

In [ ]:
# query a chromosome position
chrm_pos: int = 55_157_663
# ---------------------------
chrm_info = gA.chrm_pos_info(transcript_id, chrm_pos)
print(chrm_info)

In [ ]:
# map a chromosome position to the RNA position
chrm_pos: int = 55_211_628 
# ------------------------
if (rna_pos := gA.chrm_pos2rna_pos(transcript_id, chrm_pos)) is None:
    print(f"{chrm_pos=:,} does not overlap with RNA.")
else:
    print(f"{chrm_pos=:,} --> {rna_pos=:}")


In [ ]:
# map a RNA position to the chromosome position
rna_pos: int = 685
# ----------------
chrm_p = gA.rna_pos2chrm_pos(transcript_id, rna_pos)
print(f"{rna_pos=} --> {chrm_p=}")

In [ ]:
# query a RNA position
rna_pos: int = 685
# ----------------
chrm_info = gA.rna_pos2chrm_info(transcript_id, rna_pos)
print(chrm_info)

In [ ]:
# query an exon position
exon_number: int = 7
nt_number: int = 47
# -------------------
info = gA.exon_nt_info(transcript_id, exon_number, nt_number)
print(info)

In [ ]:
# query an exon segment
exon_number: int = 28
bp_index_in_exon: int = 400
# --------------------------
if (exon_segment_info := gA.exon_nt_segment(transcript_id, exon_number, bp_index_in_exon)) is not None:
    print(f"bp #{bp_index_in_exon} in exon #{exon_number} corresponds to bp #{exon_segment_info[1]:,} in the {exon_segment_info[0]}.")

In [ ]:
# query an amino-acid position
aa_number: int = 163
# ------------------
aa_info = gA.aa_exon_info(transcript_id, aa_number)
print(aa_info)

The `codon_exon_pos` is a list of 3 positions, each in the form `<exon number>:<pos>`.
`pos` is the 1-based position relative to the start of exon `<exon number`. The corresponding
chromosome position is then `exon_start_coordinate+m*(pos-1)`, where `m`=`1` [`m`=-`1`] for genes encoded on
the positive [negative] DNA strand (recall that in genes encoded on the negative strand, 
`exon_start_coordinate`>`exon_end_coordinate`).

In [ ]:
# query an AA variant given a DNA variant
ref_allele, var_allele, chromosome_pos = 'G', 'C', 55_152_609
# ------------------------------------------------------------
aa_var = gA.DNA_SNP_mut_to_AA_mut(ref_allele, var_allele, chromosome_pos, transcript_id)
print(f"chr{gA.chrm}:{chromosome_pos}:{ref_allele}>{var_allele} --> {aa_var=}")


In [ ]:
# query (all) DNA variants based on an AA variant
aa_var: str = 'C231S'
# -------------------
dna_all_muts = gA.AA_mut_to_DNA_SNP_mut(aa_var, transcript_id)
print(f"{aa_var=} corresponds to the following DNA variants:")
for i, (codon, var_info) in enumerate(dna_all_muts.items(), start=1):
    print(f"{i}. {codon=}, {var_info}")


## Annotating Multiple Genes
A faster instantiation of the annotation class, which is relevant when annotating multiple genes, can be done using annotation dataframes that are generated from the annotation file as shown here.

In [ ]:
# First, parse the annotation file into annotation dataframes
gff3_dfs = u.ensembl_gff3_df(annotation_full_file)  

In [ ]:
# Now instantiate with the annotation dataframes (faster)
# Can instantiate in local mode by providing chrm_fasta_file (see above)
g_a1 = u.Gene_cls('IDH1', gff3_dfs, species=species)
g_a1.info()
g_a2 = u.Gene_cls('BRCA1', gff3_dfs, species=species)
g_a2.info()
g_a3 = u.Gene_cls('EGFR', gff3_dfs, species=species)
g_a3.info()

# Annotating Other Eukaryotes Species
The default assumed species is Homo sapiens (`homo_sapiens`). To annotate other Eukaryotes species, the user must define also the annotation file signature.

The annotation file signature is identical to the annotation file name, except that the relaese number is replaced by `XXX`. For example, the signature for the annotation file of Mus musculus is `Mus_musculus.GRCm39.XXX.gff3.gz`. In general, an annotation file can be found in [Ensembl](http://www.ensembl.org/index.html) -> Select a species -> in the `Gene annotation` section select `GFF3`. 

You can get the suggested annotation file name by invoking `suggested_annotation_file_name(species=species)` as shown below.

Following is an example for annotating a `Mus musculus` gene and transcript.

In [3]:
species: str = 'mus_musculus'

# The suggest annotation file signature
if (anot_info := u.suggested_annotation_file_name(species=species)) is None:
    print(f"Error in retrieving file name. Please check manually for the GFF3 annotation file name for {species=}.")
else:
    suggest_annotation_file_name, assembly_n, release_n = anot_info
    print(f"Suggested annotation file name: {suggest_annotation_file_name}.\nTo set the annotation_file_signature, change the release numnber ({release_n}) to 'XXX': {suggest_annotation_file_name.replace(release_n, 'XXX')}")

# Annotation file signature
annotation_file_signature: str = f'{species.capitalize()}.GRCm39.XXX.gff3.gz'


Suggested annotation file name: Mus_musculus.GRCm39.113.gff3.gz.
To set the annotation_file_signature, change the release numnber (113) to 'XXX': Mus_musculus.GRCm39.XXX.gff3.gz


In [ ]:
# Informative
# Check for local Ensembl annotation files
if (local_files := u.get_annotation_files_info_in_folder(Annotation_folder, gff3_pattern=annotation_file_signature)):
    versions = list(local_files.keys())
    max_ver = max(versions)
    print(f"Found {len(local_files)} annotation files in {Annotation_folder}. {versions=}, maximal version = {max_ver}.")
else:
    print(f"No annotation file in {Annotation_folder}. Need to download from Ensembl.")

# check Ensembl version - Informative
ensembl_version = u.get_ensembl_release()
print(f"{ensembl_version=}.")

In [ ]:
enable_download: bool = False  # enables/disables annotation file download from Ensembl only when local version < ensembl version.
                               # If Annotation_folder contains no anotation file, download is done regardless of this setting.
# ------------------------------------------------------------------------------------------------------------------------------
download_done, ensembl_file, local_file = u.update_local_release_to_latest(Annotation_folder, 
                                                                           enable_download=enable_download, 
                                                                           gff3_pattern=annotation_file_signature,
                                                                           species=species)
annotation_full_file = Annotation_folder / (ensembl_file if download_done else local_file)
print(f"Annotation file to use: {annotation_full_file}.\n")

In [ ]:
# Instantiate (in remote access mode)
# Can instantiate in local mode by providing chrm_fasta_file (see above)
gene: str = 'ENSMUSG00000017167' # 'Cntnap1'  # gene name or gene ID
verbose: bool = True  # set to True to get infromation prints
# -----------------------------------------------------------------
gA = u.Gene_cls(gene, annotation_full_file, species=species, verbose=verbose)


In [ ]:
gA.info()

print(f"\n{gene} contains {len(gA)} transcripts.")
gA.show_all_transcript_IDs()

print(f"\nGene start = {gA.gene_start:,}, gene end = {gA.gene_end:,}")

In [ ]:
transcript_id: str = 'ENSMUST00000103109'
print(f"\n{transcript_id=} annotation:")

if (ts_te := gA.transcript_boundaries(transcript_id)) is None:
    raise ValueError(f"{transcript_id} is not a transcript of {gene} !!")
ts, te = ts_te
print(f"start site (TSS) = {ts:,}, end site (TES) = {te:,}.")

In [ ]:
df = gA.exon_map(transcript_id)
print(df.to_string())

In [ ]:
# protein sequence
print(f"Access mode is {gA.access_mode()}.")

aa_seq = gA.AA(transcript_id)
print(f"\nprotein:\n{aa_seq}\n{len(aa_seq):,} AAs.")